In [265]:
import pandas as pd
import os

## Reading the csv 

In [266]:
car_data = pd.read_csv('cardata.csv', index_col=False)

In [267]:
car_data

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


#### How many different types of cars do we have?


In [268]:
car_data.Car_Name.nunique()

98

In [269]:
car_data.isna().any()

Car_Name         False
Year             False
Selling_Price    False
Present_Price    False
Kms_Driven       False
Fuel_Type        False
Seller_Type      False
Transmission     False
Owner            False
dtype: bool

#### Range of years ?

In [270]:
car_data.Year.sort_values().unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018], dtype=int64)

#### Prefered Transmission

In [271]:
num_transmission = car_data.Transmission.nunique()

car_data.groupby('Transmission').agg({'Car_Name':'count'}).sort_values('Car_Name', ascending=False)

,Car_Name
Transmission,
Manual,261
Automatic,40


#### Preffered buying mode??

In [272]:
num_seller = car_data.Seller_Type.nunique()

car_data.groupby('Seller_Type').agg({'Car_Name':'count'}).sort_values('Car_Name', ascending=False)

,Car_Name
Seller_Type,
Dealer,195
Individual,106


#### Preffered fuel type ?


In [273]:
num_fuel = car_data.Fuel_Type.nunique()
#car_data.Fuel_Type.unique()
#array(['Petrol', 'Diesel', 'CNG'], dtype=object)

car_data.groupby('Fuel_Type').agg({'Car_Name':'count'}).sort_values('Car_Name', ascending=False)


,Car_Name
Fuel_Type,
Petrol,239
Diesel,60
CNG,2


#### Top 5 car based on sales

In [274]:
car_data.groupby('Car_Name').agg({'Seller_Type':'count'}).sort_values('Seller_Type', ascending=False).head(5)

,Seller_Type
Car_Name,
city,26
corolla altis,16
verna,14
fortuner,11
brio,10


In [275]:
car_data.groupby('Car_Name').agg({'Kms_Driven':'mean'}).sort_values('Kms_Driven', ascending=False).head(5)

,Kms_Driven
Car_Name,
Activa 3g,250250.0
camry,142000.0
corolla,135154.0
800,127000.0
Honda Karizma,121500.0


In [276]:
car_data.groupby('Car_Name').agg({'Seller_Type':'count', 'Selling_Price':'mean'}).sort_values('Seller_Type', ascending=False).head(5)

,Seller_Type,Selling_Price
Car_Name,,
city,26,7.419231
corolla altis,16,7.183125
verna,14,6.107857
fortuner,11,18.685455
brio,10,4.745000


#### Year and corresponding sales

In [277]:
car_data.groupby('Year').agg({'Car_Name':'count'}).sort_values('Car_Name', ascending=False)

,Car_Name
Year,
2015,61
2016,50
2014,38
2017,35
2013,33
2012,23
2011,19
2010,15
2008,7


#### Impact of inflation : What is change in price every year?


In [278]:
car_sales = car_data.groupby(['Car_Name','Year']).agg({'Selling_Price':'sum'}).sort_values(['Car_Name','Year']).reset_index()

In [279]:
car_sales['Yearly change'] = car_sales.groupby('Car_Name')['Selling_Price'].diff()
car_sales['Year Gap'] = car_sales.groupby('Car_Name')['Year'].diff()

In [280]:
car_sales['CAG'] = car_sales['Yearly change'] / car_sales['Selling_Price'].shift(1)
car_sales['CAGR%'] = (car_sales['CAG'] / car_sales['Year Gap']).mul(100)
car_sales.fillna(0, inplace=True)

In [281]:
car_sales.sort_values(['Car_Name','Year','CAGR%'], ascending=True)

,Car_Name,Year,Selling_Price,Yearly change,Year Gap,CAG,CAGR%
0,800,2003,0.35,0.00,0.0,0.000000,0.000000
1,Activa 3g,2008,0.17,0.00,0.0,0.000000,0.000000
2,Activa 3g,2016,0.45,0.28,8.0,1.647059,20.588235
3,Activa 4g,2017,0.40,0.00,0.0,0.000000,0.000000
4,Bajaj ct 100,2015,0.18,0.00,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...
201,wagon r,2013,2.90,0.05,2.0,0.017544,0.877193
202,wagon r,2015,3.25,0.35,2.0,0.120690,6.034483
203,xcent,2014,4.40,0.00,0.0,0.000000,0.000000
204,xcent,2015,4.75,0.35,1.0,0.079545,7.954545


In [282]:
car_growth = car_sales.groupby('Car_Name', as_index=False).agg({'CAGR%':'max', 'Year Gap':'sum'}).sort_values('CAGR%',ascending=False)

In [283]:
car_growth[ (car_growth['Year Gap'] >= 3) & (car_growth['CAGR%'] != 0)].sort_values('CAGR%', ascending=False)

,Car_Name,CAGR%,Year Gap
68,ciaz,380.000000,3.0
66,brio,278.301887,5.0
82,grand i10,272.435897,4.0
75,eon,232.727273,5.0
87,jazz,220.740741,7.0
69,city,205.990783,11.0
86,innova,192.771084,12.0
93,sx4,177.358491,10.0
14,Bajaj Pulsar 150,175.000000,9.0
80,etios liva,152.173913,3.0


#### Which car has the best resale market??

In [284]:
cars_resold = car_data.loc[car_data['Owner'] >=1 ]

In [285]:
cars_resold.groupby('Car_Name').agg({'Owner' : 'count'})

,Owner
Car_Name,
Bajaj Discover 125,2
Bajaj Pulsar 150,1
Hero Ignitor Disc,1
Hero Hunk,1
Hyosung GT250R,1
camry,1
etios g,1
grand i10,1
i20,1


#### Manual vs auto 

In [286]:
### which has more miles in it

cars_tranmission = car_data.groupby('Transmission', as_index=False).agg({'Kms_Driven':'mean'})

In [287]:
cars_tranmission = pd.merge(cars_tranmission,(car_data.groupby('Transmission', as_index=False).agg({'Selling_Price':'mean'})),on='Transmission',how = 'inner')

In [288]:
cars_tranmission['Selling_Price'] = cars_tranmission['Selling_Price'].round(2)

In [289]:
cars_tranmission

,Transmission,Kms_Driven,Selling_Price
0,Automatic,53062.975000,9.42
1,Manual,34477.356322,3.93


#### Key generation

In [290]:
#car_data
car_data['Key'] = car_data.apply(lambda x: '_'.join([x.Car_Name,str(x.Year)]), axis=1)

In [291]:
columns = list(car_data.columns.values)

car_data = car_data[[columns[-1]] + columns[:9]]

In [296]:
car_data.to_csv('car_analysis_output.csv', index=False)
car_sales.to_excel('car_sales_report.xlsx', index=False)
car_growth.to_csv('car_price_growth_report.txt', sep='|', index=False)